In [ ]:
import pandas as pd
import numpy as np
test=pd.read_csv("../input/test.csv")
train=pd.read_csv("../input/train.csv")

In [ ]:
submission=pd.read_csv("../input/sample_submission.csv")

In [ ]:
(train[0:10])

In [ ]:
np.shape(train)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale

In [ ]:
len(train.columns)

In [ ]:
print(train.dtypes.value_counts())
print(test.dtypes.value_counts())

In [ ]:
print(train.iloc[:,0].dtype)
test.iloc[:,0].dtype

In [ ]:
pca_frame=scale(train.iloc[:,2:].values)

In [ ]:
pca_frame_test=scale(test.iloc[:,1:].values)

In [ ]:
print(np.shape(pca_frame))
np.shape(pca_frame_test)

In [ ]:
pca=PCA(n_components=2000)

In [ ]:
pca.fit(pca_frame)

In [ ]:
var= pca.explained_variance_ratio_

In [ ]:
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

In [ ]:
plt.plot(var1)
plt.show()

In [ ]:
pca_frame1=pca.fit_transform(pca_frame)
np.shape(pca_frame1)

In [ ]:
pca_frame1_test=pca.transform(pca_frame_test)

In [ ]:
df=pd.DataFrame(pca_frame1)

In [ ]:
df1=pd.DataFrame(pca_frame1_test)

In [ ]:
np.shape(df)

In [ ]:
np.shape(df1)

In [ ]:
df[0:10]

In [ ]:
df1[0:10]

In [ ]:
final_train=pd.concat([train.iloc[:,:1],df],axis=1)
print(final_train[0:10])
np.shape(final_train)

In [ ]:
target=train.iloc[:,1]
print(target)
print(np.shape(target))

In [ ]:
final_test=pd.concat([test.iloc[:,:1],df1],axis=1)

In [ ]:
print(np.shape(final_test))
print(np.shape(final_train))
print(np.shape(target))

In [ ]:
X_train = final_train.drop(["ID"], axis=1)
y_train = np.log1p(target.values)

X_test = final_test.drop(["ID"], axis=1)

In [ ]:
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))

In [ ]:
from sklearn.model_selection import train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'objective': 'reg:linear', 
          'eval_metric': 'rmse',
          'eta': 0.005,
          'max_depth': 15, 
          'subsample': 0.7, 
          'colsample_bytree': 0.5,
          'alpha':0,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=100)
    
    dtest = xgb.DMatrix(test_X)
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

#import lightgbm as lgb
import xgboost as xgb
#from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
print("XGB Training Completed...")
sub_xgb = pd.DataFrame()
sub_xgb["target"] = pred_test_xgb

In [ ]:
submission["target"]=sub_xgb["target"]
print(submission.head())
submission.to_csv('sub_lgb.csv', index=False)